Handle Imports

In [ ]:
import sys
import os
import ast
import argparse
import pandas as pd
import plotly.graph_objs as go
pd.options.plotting.backend = "plotly"
from logging_plotter import parse_datastamp_topic_csv


MACROS

In [ ]:
VICON_FILE = '/home/will/drl_swarm/data_analysis/vicon_outputs.csv'
LH_FILE = '/home/will/drl_swarm/data_analysis/lh_data.csv'
ODOM_FILE = '/home/will/drl_swarm/data_analysis/odom_data.csv'

 Read in vicon outputs

In [ ]:
vicon_topics = {"/rigid_bodies.rigidbodies[0].pose.position.x": "tb1_x",
                "/rigid_bodies.rigidbodies[0].pose.position.y": "tb1_y",
                "/rigid_bodies.rigidbodies[0].pose.position.z": "tb1_z",
                "/rigid_bodies.rigidbodies[1].pose.position.x": "tb2_x",
                "/rigid_bodies.rigidbodies[1].pose.position.y": "tb2_y",
                "/rigid_bodies.rigidbodies[1].pose.position.z": "tb2_z",
                "/rigid_bodies.rigidbodies[2].pose.position.x": "cf_x",
                "/rigid_bodies.rigidbodies[2].pose.position.y": "cf_y",
                "/rigid_bodies.rigidbodies[2].pose.position.z": "cf_z",}
odom_topics = {"/odom.pose.pose.position.x": "odom_x",
                "/odom.pose.pose.position.y": "odom_y",
                "/odom.pose.pose.position.z": "odom_z",}
lh_topics = {'/tf.transforms[:]{child_frame_id=="lighthouse_link"}.transform.translation.x': 'lh_x',
                '/tf.transforms[:]{child_frame_id=="lighthouse_link"}.transform.translation.y': 'lh_y',
                '/tf.transforms[:]{child_frame_id=="lighthouse_link"}.transform.translation.z': 'lh_z',}

In [ ]:
df_vicon = parse_datastamp_topic_csv(VICON_FILE, vicon_topics)
df_lh = parse_datastamp_topic_csv(LH_FILE, lh_topics)
df_odom = parse_datastamp_topic_csv(ODOM_FILE, odom_topics)
tb_vicon_data = df_vicon.filter(like="tb")
cf_vicon_data = df_vicon.filter(like="cf")

Simple Plot Outputs

In [ ]:
fig1 = tb_vicon_data.plot(title="TB Vicon Data", x =tb_vicon_data.index, y=tb_vicon_data.columns, kind = 'scatter', template = 'plotly_dark')
fig1.show(renderer="browser")

fig2 = cf_vicon_data.plot(title="CF Vicon Data", x =cf_vicon_data.index, y=cf_vicon_data.columns, kind = 'scatter', template = 'plotly_dark')
fig2.show(renderer="browser")

fig3 = df_lh.plot(title="Lighthouse Data", x =df_lh.index, y=df_lh.columns, kind = 'scatter', template = 'plotly_dark')
fig3.show(renderer="browser")

fig4 = df_odom.plot(title="Odometry Data", x =df_odom.index, y=df_odom.columns, kind = 'scatter', template = 'plotly_dark')
fig4.show(renderer="browser")


# Start of Data Analysis and Cleaning

### Initial Positions

Get offsets for crazyflie lh and vicon initial positions

In [ ]:
vicon_cf_offset = (cf_vicon_data.dropna()).head(75).mean()
lh_cf_offset = (df_lh.dropna()).head(75).mean()
print(f"vicon_cf_offset: {vicon_cf_offset}")
print(f"lh_cf_offset: {lh_cf_offset}")

Get offsets for the turtlebot positions

In [ ]:
print(tb_vicon_data.mean())


## Compare Crazyflie position when computed from Vicon and Lighthouse

In [ ]:
comparison_df = pd.concat([df_vicon.filter(like="cf"), df_lh], axis=1)
comparison_df.dropna(axis=0, how='all', inplace=True)
fig5 = comparison_df.plot(title="Comparison plot", x = comparison_df.index, y = comparison_df.columns, kind = "scatter", template="plotly_dark")
fig5.show(renderer="browser")

Subtract initial position from `comparison_df`

In [ ]:
comparison_df_normalized = comparison_df - pd.concat([vicon_cf_offset, lh_cf_offset])
comparison_df_x = comparison_df_normalized.filter(like="_x")
comparison_df_y = comparison_df_normalized.filter(like="_y")
comparison_df_z = comparison_df_normalized.filter(like="_z")

In [ ]:
fig6 = comparison_df_x.plot(title="Vicon vs LH x", x = comparison_df_x.index, y = comparison_df_x.columns, kind = "scatter", template="plotly_dark")
fig6.show(renderer="browser")

fig7 = comparison_df_y.plot(title="Vicon vs LH y", x = comparison_df_y.index, y = comparison_df_y.columns, kind = "scatter", template="plotly_dark")
fig7.show(renderer="browser")

fig8 = comparison_df_z.plot(title="Vicon vs LH z", x = comparison_df_z.index, y = comparison_df_z.columns, kind = "scatter", template="plotly_dark")
fig8.show(renderer="browser")
